# ADS Capstone - Week 3 Project
## Scrape and Clean

In [1]:
#! pip3 install beautifulsoup4
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).text
soup = BeautifulSoup(page, 'lxml')
#print(soup.prettify())

## Split header and rows

In [3]:
body = soup.findAll('table')[0].findAll('tr') #generate list of rows for html table
head=body[0] #first row is column titles  
body_rows = body[1:]

In [4]:
headings = []
for item in head.find_all('th'):
    item = (item.text).rstrip("\n") #removes extra charaters from header 
    headings.append(item)
print(headings)

['Postal Code', 'Borough', 'Neighbourhood']


In [5]:
rows_all = []
for row_num in range (len(body_rows)):
    row = []
    for row_item in body_rows[row_num].find_all('td'):
        aa = re.sub('(\n)','',row_item.text)
        row.append(aa)
    rows_all.append(row)
df_tor = pd.DataFrame(rows_all, columns = headings)

## Remove Borough Not Assigned from Data Frame

In [6]:
df_tor = df_tor[~df_tor.Borough.str.contains('Not assigned')].reset_index(drop = True)
df_tor.shape

(103, 3)

## Determining Geographical Coordinates of Postal Codes

In [7]:
#! pip3 install geocoder
import geocoder # import geocoder

In [8]:
latitude=[]
longitude=[]
for code in df_tor['Postal Code']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

In [9]:
df_tor['Latitude'] = latitude
df_tor['Longitude'] = longitude
df_tor.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


## Neighbourhood Clustering

In [23]:
#!conda install -c conda-forge geopy --yes
#! pip install folium

In [10]:
import json 
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [11]:
# Subsetting data for Old City of Toronto only 
toronto_data = df_tor[df_tor.Borough.str.contains('Toronto')].reset_index(drop=True)
toronto_data.shape

(39, 5)

In [12]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
# create map of Toronto with Neighoods by postal codes and labeling by neighbourhood
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Genrerate Venue List from Foursquare

In [14]:
# Function to obtain nearby Venues by Postal Code

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
CLIENT_ID = '' 
CLIENT_SECRET = '' 
VERSION = '20180605'
LIMIT = 100

In [16]:
# Generate Dataframe of Venues by Postal Code
toronto_venues = getNearbyVenues(names=toronto_data['Postal Code'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

M5A
M7A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


In [17]:
# One hot encoding of Venue Groups and calulate means of each Group
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped.head()

,Postal Code,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,M4K,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,M4L,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,M4M,0.0,0.0,0.039216,0.019608,0.0,0.0,0.0,0.039216,0.0,...,0.00,0.0,0.0,0.0,0.0,0.019608,0.0,0.0,0.0,0.0
4,M4N,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [18]:
kclusters = 7
toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:5]

array([6, 0, 0, 0, 2], dtype=int32)

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
# Generate Most Common Venues for each Postal Code
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Pub,Trail,Neighborhood,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,M4K,Bus Line,Ice Cream Shop,Coffee Shop,Discount Store,Grocery Store,Intersection,Park,Business Service,Fish & Chips Shop,Fast Food Restaurant
2,M4L,Park,Italian Restaurant,Pet Store,Pizza Place,Pub,Coffee Shop,Restaurant,Movie Theater,Sandwich Place,Fast Food Restaurant
3,M4M,Italian Restaurant,Diner,Brewery,Pizza Place,Gastropub,Sushi Restaurant,American Restaurant,Coffee Shop,Arts & Crafts Store,Bar
4,M4N,Bus Line,Swim School,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [21]:
# Merge Dataframes of Neighhourhood data, Venue Data, and Clustering

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postal Code'), on = 'Postal Code', how = 'right')
toronto_merged.head() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0,Coffee Shop,Breakfast Spot,Theater,Spa,Food Truck,Event Space,Electronics Store,Restaurant,Bakery,Italian Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,0,Coffee Shop,Sandwich Place,Gastropub,Theater,Burrito Place,Café,Falafel Restaurant,Fried Chicken Joint,Bank,Italian Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,0,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Cosmetics Shop,Hotel,Chinese Restaurant,Diner,Tanning Salon,Theater
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,0,Coffee Shop,Cocktail Bar,Clothing Store,Hotel,Gastropub,Restaurant,Café,Cosmetics Shop,Beer Bar,Lingerie Store
4,M4E,East Toronto,The Beaches,43.67709,-79.29547,6,Health Food Store,Pub,Trail,Neighborhood,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [22]:
# Generate map of Neighboorhood clusters

colors_out = ['black', 'red', 'darkorchid','fuchsia','darkred','blue']
colors_in = ['grey','firebrick','darkorange','darkgreen','lime', 'orange']
# create map
toronto_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors_out[cluster-1],
        fill=True,
        fill_color=colors_in[cluster-1],
        fill_opacity=0.7).add_to(toronto_map_clusters)
       
toronto_map_clusters